In [1]:
import datetime
import locale
import os
import pandas as pd
import tweepy
import re
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from dotenv import load_dotenv

In [2]:
locale.setlocale( locale.LC_ALL, 'en_US.UTF-8' ) 

'en_US.UTF-8'

In [3]:
os.chdir('/Users/aurorapeng/Documents/GitHub/Luxury-hotel-sentiment-analysis')
path_to_driver_1 = '/Users/aurorapeng/Documents/GitHub/Luxury-hotel-sentiment-analysis/chromedriver'

# Scrape Tripadvisor

In [4]:
def scrape_tripadvisor_review(path_to_driver_1,link_to_scrape):
    driver = webdriver.Chrome(executable_path=path_to_driver_1)

    # %%%% Link
    driver.get(link_to_scrape)

    WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.XPATH, "//span[@class='iypZC Mc _R b']")))

    total_n_reviews_int = driver.find_element("xpath","//span[@class='iypZC Mc _R b']").text

    total_n_reviews_int = int(total_n_reviews_int)-220


    # %%%% Scraping
    reviews_dict = []
    while True:
        # waits for the reviews to be visible before grabbing their elements
        WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.XPATH, "//div[@class='YibKl MC R2 Gi z Z BB pBbQr']")))


        reviews = driver.find_elements("xpath","//div[@class='YibKl MC R2 Gi z Z BB pBbQr']")

        r=0
        # for loop to scrape through reviews
        for r in range(len(reviews)):
            one_review                     = {}
            one_review['scrapping_date']   = datetime.datetime.now()
            try:                
                soup                       = BeautifulSoup(reviews[r].get_attribute('innerHTML'))
            except:
                    # I got StaleElementReferenceException a lot here, so in order to solve this problem,
                    # I just refresh the page, make sure the elements are there and reinitialize reviews.
                WebDriverWait(driver, 10).until(
                    EC.visibility_of_element_located((By.XPATH, "//div[@class='YibKl MC R2 Gi z Z BB pBbQr']")))
                reviews = driver.find_elements("xpath","//div[@class='YibKl MC R2 Gi z Z BB pBbQr']")
                soup                       = BeautifulSoup(reviews[r].get_attribute('innerHTML'))


            # pulls reviwerid and date from innerHTML soup
            try:
                one_review_id            = soup.find('div', attrs={'class':'cRVSd'}).text.split(' wrote a review ')[0].strip()
                one_reviewer_date        = soup.find('div', attrs={'class':'cRVSd'}).text.split(' wrote a review ')[1].strip()
            except:
                one_review_id            = ""
                one_reviewer_date        = ""
            one_review['reviewer_name']  = one_review_id
            one_review['review_date']    = one_reviewer_date

            # pulls rating from innerHTML soup
            try:
                for headlines in soup.findAll('div',{'class':'Hlmiy F1'}):
                    for p in headlines.find_all('span', recursive=False):
                        ratings = int(p['class'][1].replace('bubble_','').strip())/10
            except:
                ratings            = ""
            one_review['rating']      = ratings

            # pulls review title from innerHTML soup
            try:
                one_review_title            = soup.find('div', attrs={'class':'KgQgP MC _S b S6 H5 _a'}).text.strip()
            except:
                one_review_title            = ""
            one_review['review_title']  = one_review_title

            # pulls review from innerHTML soup
            try:
                one_review_content            = soup.find('q', attrs={'class':'QewHA H4 _a'}).text.strip()
            except:
                one_review_title            = ""
            one_review['review_content']  = one_review_content

            # pulls date of stay from innerHTML soup
            try:
                date_of_stays            = soup.find('span', attrs={'class':'teHYY _R Me S4 H3'}).text.replace('Date of stay: ','').strip()
            except:
                date_of_stays            = ""
            one_review['date_of_stays']  = date_of_stays

            reviews_dict.append(one_review)

        # Keeps clicking next page until all reviews are collected
        #if len(reviews_dict) < total_n_reviews_int: The tricky thing is in the Tripadvisor, the ACTUAL review number is actually less than what they propaganed in their website, 
        # which always tell me StaleElementReferenceException, in order to solve this question I just got to the final page which is the 32rd page and count there are only 3 views left, 
        #which makes the actual total review only 3103 views, we can assume maybe there are 200 views that are not diclosed
        if len(reviews_dict) < total_n_reviews_int:
            # clicks for next page
            try:
                # waits for the next page button to be visible, then clicks
                WebDriverWait(driver, 10).until(
                    EC.visibility_of_element_located((By.XPATH, "//a[@class='ui_button nav next primary ']")))
                driver.find_element("xpath","//a[@class='ui_button nav next primary ']").click()
            except:

                # then clicks next page
                driver.find_element("xpath","//a[@class='ui_button nav next primary ']").click()
        # When all reviews are in the dictionary, the while(True) loop breaks
        else:
            break 
    driver.close()
    return reviews_dict

## Scrape the Tier One Hotel: Waldorf

In [5]:
link_to_scrape = 'https://www.tripadvisor.in/Hotel_Review-g35805-d1516481-Reviews-Waldorf_Astoria_Chicago-Chicago_Illinois.html'

In [ ]:
waldorf_reviews = pd.DataFrame.from_dict(scrape_tripadvisor_review(path_to_driver_1,link_to_scrape))

In [ ]:
file_name = 'waldorf_reviews_tripadvisor.csv'
waldorf_reviews.to_csv(file_name)

## Scrape the Tier Two Hotel: DoubleTree by Hilton Chicago

In [8]:
link_to_scrape = 'https://www.tripadvisor.com/Hotel_Review-g35805-d1235890-Reviews-TheWit_Chicago_a_DoubleTree_by_Hilton_Hotel-Chicago_Illinois.html'
doubletree_reviews = pd.DataFrame.from_dict(scrape_tripadvisor_review(path_to_driver_1,link_to_scrape))
file_name = 'doubletree_reviews_tripadvisor.csv'
doubletree_reviews.to_csv(file_name)

/var/folders/4l/0fc5xvvd01g47wjlznw52ghm0000gn/T/ipykernel_90443/4290313188.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=path_to_driver_1)


## Scrape the Tier Three Hotel : Hilton Garden Inn Chicago

In [9]:
link_to_scrape = 'https://www.tripadvisor.com/Hotel_Review-g35805-d111487-Reviews-Hilton_Garden_Inn_Chicago_Downtown_Magnificent_Mile-Chicago_Illinois.html'
hiltongarden_reviews = pd.DataFrame.from_dict(scrape_tripadvisor_review(path_to_driver_1,link_to_scrape))
file_name = 'hiltongarden_reviews_tripadvisor.csv'
hiltongarden_reviews.to_csv(file_name)

/var/folders/4l/0fc5xvvd01g47wjlznw52ghm0000gn/T/ipykernel_90443/4290313188.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=path_to_driver_1)


# Scrape Expedia

In [23]:
def open_up_driver(link_to_scrape):
    driver = webdriver.Chrome(executable_path=path_to_driver_1)

    # %%%% Link
    driver.get(link_to_scrape)
    return driver

In [24]:
def open_up_expedia_first(driver):

    WebDriverWait(driver, 3).until(
            EC.visibility_of_element_located((By.XPATH, "//button[@data-stid='reviews-link']")))

    driver.find_element("xpath","//button[@data-stid='reviews-link']").click()
    

In [63]:
def expedia_review_scrape(driver):
    
    WebDriverWait(driver, 3).until(
            EC.visibility_of_element_located((By.XPATH, "//button[@class='uitk-link uitk-spacing uitk-spacing-padding-blockstart-two uitk-link-align-left uitk-link-layout-default uitk-link-medium']")))

    driver.find_element("xpath","//button[@class='uitk-link uitk-spacing uitk-spacing-padding-blockstart-two uitk-link-align-left uitk-link-layout-default uitk-link-medium']").click()

    WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.XPATH, "//div[@class='uitk-text uitk-text-spacing-one uitk-type-300 uitk-text-default-theme']")))

    total_n_reviews_raw = driver.find_element("xpath","//div[@class='uitk-text uitk-text-spacing-one uitk-type-300 uitk-text-default-theme']").text
    total_n_reviews_number = total_n_reviews_raw.strip(' verified reviews')
    total_n_reviews_int = locale.atoi(total_n_reviews_raw.strip(' verified reviews'))
    click_times = total_n_reviews_int//10
    for r in range(click_times):
        # clicks for next page
        try:
            # waits for the next page button to be visible, then clicks
            WebDriverWait(driver, 3).until(
                EC.visibility_of_element_located((By.XPATH, "//div[@class='uitk-spacing uitk-type-center uitk-spacing-margin-block-three']")))
            driver.find_element("xpath","//div[@class='uitk-spacing uitk-type-center uitk-spacing-margin-block-three']").click()

        except:
     # waits for the next page button to be visible, then clicks
            WebDriverWait(driver, 3).until(
                EC.visibility_of_element_located((By.XPATH, "//div[@class='uitk-spacing uitk-type-center uitk-spacing-margin-block-three']")))
            # then clicks next page
            driver.find_element("xpath","//div[@class='uitk-spacing uitk-type-center uitk-spacing-margin-block-three']").click()

    reviews_dict = []
    WebDriverWait(driver, 3).until(
        EC.visibility_of_element_located((By.XPATH, "//div[@class='uitk-card-content-section uitk-card-content-section-border-block-end uitk-card-content-section-padded']")))

    reviews = driver.find_elements(By.XPATH, "//div[@class='uitk-card-content-section uitk-card-content-section-border-block-end uitk-card-content-section-padded']")
    r=0
    # for loop to scrape through reviews
    for r in range(len(reviews)):
        one_review                     = {}
        one_review['scrapping_date']   = datetime.datetime.now()
        try:                
            soup                       = BeautifulSoup(reviews[r].get_attribute('innerHTML'))
        except:
                # I got StaleElementReferenceException a lot here, so in order to solve this problem,
                # I just refresh the page, make sure the elements are there and reinitialize reviews.
            WebDriverWait(driver, 3).until(
                EC.visibility_of_element_located((By.XPATH, "//article[@itemprop='review']")))
            reviews = driver.find_elements("xpath","//article[@itemprop='review']")
            soup                       = BeautifulSoup(reviews[r].get_attribute('innerHTML'))


        # pulls reviwer rating from innerHTML soup
        try:
            one_review_rating           = soup.find('span', attrs={'itemprop':'ratingValue'}).text.strip()
        except:
            one_review_rating           = ""
        one_review['review_rating']  = one_review_rating

        # pulls reviewer name from innerHTML soup
        try:
            one_reviewer_name           = soup.find('h5', attrs={'class':'uitk-heading uitk-heading-7'}).text.strip()
        except:
            one_reviewer_name            = ""
        one_review['reviewer_name']      = one_reviewer_name

        # pulls review date from innerHTML soup
        try:
            one_reviewer_date           = soup.find('span', attrs={'itemprop':'datePublished'}).text.strip()
        except:
            one_reviewer_date            = ""
        one_review['review_date']  = one_reviewer_date

        # pulls review title from innerHTML soup
        try:
            one_review_content_title            = soup.find('h5', attrs={'class':'uitk-heading uitk-heading-6'}).text.strip()
        except:
            one_review_content_title            = ""
        one_review['review_title']  = one_review_content_title

        # pulls review content from innerHTML soup
        try:
            one_review_content            = soup.find('span', attrs={'itemprop':'description'}).text.strip()
        except:
            one_review_content            = ""
        one_review['review_content']  = one_review_content

        reviews_dict.append(one_review)
    driver.close()
    return reviews_dict

## Scrape the Tier One Hotel: Waldorf

In [59]:
link_to_scrape = 'https://www.expedia.com/Chicago-Hotels-Waldorf-Astoria-Chicago.h2717582.Hotel-Information?chkin=2022-11-17&chkout=2022-11-18&destType=MARKET&destination=Chicago%2C%20Illinois%2C%20United%20States%20of%20America&neighborhoodId=553248635976470100&pwa_ts=1667488480957&referrerUrl=aHR0cHM6Ly93d3cuZXhwZWRpYS5jb20vSG90ZWwtU2VhcmNo&regionId=829&rfrr=HSR&rm1=a2&selected=2717582&selectedRatePlan=222049559&selectedRoomType=201371071&sort=RECOMMENDED&top_cur=USD&top_dp=548&useRewards=false&userIntent=&x_pwa=1'

In [61]:
driver = open_up_driver(link_to_scrape)

/var/folders/4l/0fc5xvvd01g47wjlznw52ghm0000gn/T/ipykernel_90443/3466519808.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=path_to_driver_1)


In [64]:
#refresh, do the rest of the scraping
reviews_dict = expedia_review_scrape(driver)

In [65]:
# %%%% To excel!
expedia_waldorf_reviews = pd.DataFrame.from_dict(reviews_dict)
file_name = 'expedia_waldorf_reviews.csv'
expedia_waldorf_reviews.to_csv(file_name)

## Scrape the Tier Two Hotel: DoubleTree by Hilton Chicago

In [66]:
link_to_scrape = 'https://www.expedia.com/Chicago-Hotels-DoubleTree-By-Hilton-Chicago-Magnificent-Mile.h21152.Hotel-Information?chkin=2022-11-17&chkout=2022-11-18&x_pwa=1&rfrr=HSR&pwa_ts=1667529187230&referrerUrl=aHR0cHM6Ly93d3cuZXhwZWRpYS5jb20vSG90ZWwtU2VhcmNo&useRewards=false&rm1=a2&destination=300+E+Ohio+St%2C+Chicago%2C+IL+60611%2C+USA&destType=ADDRESS&neighborhoodId=6130109&trackingData=AAAAEAB-BIgTf5G1NhzCaLgJTaWrYd9NlGvRQ6QytNZWZVX7nmMtD6_KrooBdgUjqDsyUC_hysxNoRWZpsOmqYpqH1pwDj14DFR6cWZg4sPDuAC0igupTFMrGOxew6nR7PqLQshXFahWGXVnbbJpTv6RrxYVpZWZXOcuUA2KFDxyTQG84MAXyQ9eON-eCjv8UbZ8nM3pmBp4B9Jg_2QITLSbF_eEGfMkLm50IP9VbbW7zOlXVVU6AkQrvfUqinvwKME6dYYt3uhBs_nnAL9zcyXSs76Qp57y9UKQ-xtQjtaknTzabpIQlz6cTOGyWiP7qcJ7IgdshQWoaw89gVA1XdDcJQntF4sb0UPN_UzjO9bc1ZUMb8qCYRQMUQ1lIIK3byD0WkTe8jochuUA2V6bceVlaBm5SQglFd1zQ420ufFlpMchPVRjLzvwYsn5a8J-rlXQ91C5VAwhrFnRxieT2tXxS-WtRqPULCVPDU4dA6AFZfRp_ALCBdy4pCnZr-H42uFoXeoxePunRG7NFKBlJueLsihC_CMrBngkJ3L2zLOUQnPAaoRmlDPpeeg8tIA2zI1Jnq9pYQMeANcUH1UFwoNk_34jnc1HtFhMP0Gsq-LVjB9uPq3GrtHKMzJjMTuAsfQXzgrn3-JsJqeYmmXGNC8FcZs6dsoImdoERDrwYlPrSAaO7I5A1osDZ1SqY_fsYLTyv7htcddemx7bVOcwcp3kTH2PMeWXtw72_1NI_FSEBRsfGNMUzNIN2qQDaVYvTmEw721vDXmeQFrUwtF4KYq0wULig249sELmlh0UAURTkIRnyVmo2vogzaRbJfa3a4XC1SfUssfAEYEJR2oNpl3sNg8314YxnMLVblQOC7Z13bssjRQTAhhCv6C1uByme3a8kszfjjmf_bH9ng57FfBNyktHq5ikYJwm5HTehZ5n8JCaasfUmMBOncMVWv4h5dwdJQNza_-1QzeFl3OdoyRBvZj7KUXEIi0XqSj51ILEss0XVVba4jfuUw79fFGz6VtPHxG9qMVgxUX-kXreKonTGRGzeD56jC0_URSjkSJL-ccCYlN8r2wR_iYRV4-_DwHpINneDEjs8khKpOtHX34I97DwyfpuPXc12-mAGY73_erg&rank=1&testVersionOverride=Buttercup%2C44204.0.0%2C44203.0.0%2C43549.129874.3%2C43550.132977.0%2C31936.102311.0%2C33775.98848.1%2C38414.114301.0%2C39483.0.0%2C38427.115718.1%2C42444.0.0%2C42589.0.0%2C42876.124673.0%2C42973.0.0%2C42974.0.0%2C42975.0.0%2C42976.0.0%2C42802.125960.1%2C33739.99567.0%2C37898.109354.0%2C37930.0.0%2C37949.0.0%2C37354.113955.1%2C43435.128144.0%2C44597.132747.1%2C44701.0.0&slots=&position=1&beaconIssued=2022-11-04T02%3A33%3A06&sort=RECOMMENDED&top_dp=152&top_cur=USD&userIntent=&selectedRoomType=272309&selectedRatePlan=252035191'

In [67]:
driver = open_up_driver(link_to_scrape)

/var/folders/4l/0fc5xvvd01g47wjlznw52ghm0000gn/T/ipykernel_90443/3466519808.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=path_to_driver_1)


In [68]:
#refresh, do the rest of the scraping
reviews_dict = expedia_review_scrape(driver)
expedia_doubletree_reviews = pd.DataFrame.from_dict(reviews_dict)

In [69]:
# %%%% To excel!
file_name = 'expedia_doubletree_reviews.csv'
expedia_doubletree_reviews.to_csv(file_name)

## Scrape the Tier Three Hotel : Hilton Garden Inn Chicago

In [71]:
link_to_scrape = 'https://www.expedia.com/Chicago-Hotels-Hilton-Garden-Inn-Chicago-Downtown-Riverwalk.h11501235.Hotel-Information?chkin=2022-11-17&chkout=2022-11-18&x_pwa=1&rfrr=HSR&pwa_ts=1667529399393&referrerUrl=aHR0cHM6Ly93d3cuZXhwZWRpYS5jb20vSG90ZWwtU2VhcmNo&useRewards=false&rm1=a2&destination=300+E+Ohio+St%2C+Chicago%2C+IL+60611%2C+USA&destType=ADDRESS&neighborhoodId=651190586167894016&hotelName=hilton+garden+inn&sort=RECOMMENDED&top_dp=183&top_cur=USD&userIntent=&selectedRoomType=201165573&selectedRatePlan=278102663'
driver = open_up_driver(link_to_scrape)

/var/folders/4l/0fc5xvvd01g47wjlznw52ghm0000gn/T/ipykernel_90443/3466519808.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=path_to_driver_1)


In [72]:
#refresh, do the rest of the scraping
reviews_dict = expedia_review_scrape(driver)
expedia_hiltongarden_reviews = pd.DataFrame.from_dict(reviews_dict)

In [73]:
# %%%% To excel!
file_name = 'expedia_hiltongarden_reviews.csv'
expedia_hiltongarden_reviews.to_csv(file_name)

## scrape tweeter

The aim for this dataset is to scraping the twitter's raw comment for the brand Caraway. We are sure that for the Caraway brand as a cookware brand they are also curious about how they are being talked about online.

## 1.Set up the environment

In [2]:
#Input the twitter Dev
consumer_key = "h6Krb4Ho5LpGVIdYsCIbDjFXI"
consumer_secret = "suroSy8MRGuevHO9pStnKdZm4WJMTTnf7NXdr85ueEgLLESRis"
access_token = "1471011582377447425-69XGviuJUqtGJ9TbTzBLEjS2TM5jWc"
access_token_secret = "KgqqBL42JaVFhnGag3UuJgJxtW6HSQAikYx4v0Yhr7sjp"

# Send authorization to Twitter API
auth = tweepy.OAuth1UserHandler(
  consumer_key, 
  consumer_secret, 
  access_token, 
  access_token_secret
)

api = tweepy.API(auth)

## 2.Scrape Waldorf Astoria tweets

In [5]:
text_query = 'Waldorf Astoria'
count = 100
try:
 # Creation of query method using parameters
 tweets = tweepy.Cursor(api.search_tweets,q=text_query).items(count)
 
 # Pulling information from tweets iterable object
 tweets_list = [[tweet.created_at, tweet.id, tweet.text] for tweet in tweets]
 
 # Creation of dataframe from tweets list
 # Add or remove columns as you remove tweet information
 tweets_df = pd.DataFrame(tweets_list)
 
except BaseException as e:
    print('failed on_status,',str(e))
    time.sleep(3)

file_name = 'Waldorf_tweets.csv'
tweets_df.to_csv(file_name)

## 3. Scrape DoubleTree by Hilton Chicago tweets

In [6]:
text_query = 'DoubleTree by Hilton Chicago'
count = 100
try:
 # Creation of query method using parameters
 tweets = tweepy.Cursor(api.search_tweets,q=text_query).items(count)
 
 # Pulling information from tweets iterable object
 tweets_list = [[tweet.created_at, tweet.id, tweet.text] for tweet in tweets]
 
 # Creation of dataframe from tweets list
 # Add or remove columns as you remove tweet information
 tweets_df = pd.DataFrame(tweets_list)
 
except BaseException as e:
    print('failed on_status,',str(e))
    time.sleep(3)

file_name = 'DoubleTree_by_Hilton_Chicago_tweets.csv'
tweets_df.to_csv(file_name)

## 4. Scrape Hilton Garden Inn tweets

In [7]:
text_query = 'Hilton_Garden_Inn'
count = 100
try:
 # Creation of query method using parameters
 tweets = tweepy.Cursor(api.search_tweets,q=text_query).items(count)
 
 # Pulling information from tweets iterable object
 tweets_list = [[tweet.created_at, tweet.id, tweet.text] for tweet in tweets]
 
 # Creation of dataframe from tweets list
 # Add or remove columns as you remove tweet information
 tweets_df = pd.DataFrame(tweets_list)
 
except BaseException as e:
    print('failed on_status,',str(e))
    time.sleep(3)

file_name = 'Hilton_Garden_Inn_tweets.csv'
tweets_df.to_csv(file_name)

# Data Wrangling and Data Cleaning

To combine data from two travel website sources into a master table, some preliminary data cleaning were required. For consistency, I 
(1) renamed some variable names, 
(2) limited review ratings to one digit (with no decimal places or word descriptions like "Excellent"), 
(3) added a column indicating the data source (i.e. Tripadvisor or Expedia), 
(4) added a column indicating the hotel name (i.e. Waldorf, DoubleTree, or Hilton Garden Inn), 
(5) added a column indicating the time period (i.e. pre-Covid, during-Covid, or post-Covid), and 
(6) rescaled rating from a 0-5 scale to a 0-1 scale.

In [ ]:
# Waldorf
# Tripadvisor
waldorf_reviews['hotel_name'] = 'Waldorf'
waldorf_reviews['source_name'] = 'Tripadvisor'
# Expedia
expedia_waldorf_reviews['hotel_name'] = 'Waldorf'
expedia_waldorf_reviews['source_name'] = 'Expedia'
expedia_waldorf_reviews = expedia_waldorf_reviews.rename(columns={'review_rating': 'rating'})
expedia_waldorf_reviews['rating'] = expedia_waldorf_reviews['rating'].str[0]
expedia_waldorf_reviews['rating'].astype(float)

# DoubleTree by Hilton Chicago
# Tripadvisor
doubletree_reviews['hotel_name'] = 'DoubleTree'
doubletree_reviews['source_name'] = 'Tripadvisor'
# Expedia
expedia_doubletree_reviews['hotel_name'] = 'DoubleTree'
expedia_doubletree_reviews['source_name'] = 'Expedia'
expedia_doubletree_reviews = expedia_doubletree_reviews.rename(columns={'review_rating': 'rating'})
expedia_doubletree_reviews['rating'] = expedia_doubletree_reviews['rating'].str[0]
expedia_doubletree_reviews['rating'].astype(float)

# Hilton Garden Inn Chicago
# Tripadvisor
hiltongarden_reviews['hotel_name'] = 'Hilton Garden Inn'
hiltongarden_reviews['source_name'] = 'Tripadvisor'

# Expedia
expedia_hiltongarden_reviews['hotel_name'] = 'Hilton Garden Inn'
expedia_hiltongarden_reviews['source_name'] = 'Expedia'
expedia_hiltongarden_reviews = expedia_hiltongarden_reviews.rename(columns={'review_rating': 'rating'})
expedia_hiltongarden_reviews['rating'] = expedia_hiltongarden_reviews['rating'].str[0]
expedia_hiltongarden_reviews['rating'].astype(float)

# Combine all datasets and export
master_table = pd.concat([waldorf_reviews, 
                          expedia_waldorf_reviews, 
                          doubletree_reviews, 
                          expedia_doubletree_reviews, 
                          hiltongarden_reviews, 
                          expedia_hiltongarden_reviews])

# Define the following periods:
# post-Covid period as Year > 2020, during-Covid as Year = 2020, pre-Covid as Year < 2020
period = []
for i in master_table['review_date']:
    if (i.endswith('2020') == True):
        x = 'during-Covid'    
        period.append(x)
    elif (i.endswith(('2021', '2022'))):     
        x = 'post-Covid'
        period.append(x)
    else:    
        x = 'pre-Covid'
        period.append(x)
master_table['period'] = period

# scale rating to 0-1
master_table['rating_rescale'] = master_table['rating'].astype(int)/5
master_table['review_content'].astype(str)


master_table.to_csv(r'clean_data/master_table.csv', header=True)